In [54]:
import pandas as pd

# Load the CSV file into a DataFrame
embedded_df = pd.read_csv('embedded_review_reduced150.csv',on_bad_lines='skip')

# Display the first few rows of the DataFrame to verify it was loaded correctly
embedded_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label,...,140,141,142,143,144,145,146,147,148,149
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8.0,27-Apr-10,192.0,712.0,2.0,4.0,...,0.028273,0.953118,-0.814256,1.995593,0.972909,-0.869742,-0.927267,1.195638,-0.330614,0.845055
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5.0,14-Dec-09,17.0,708.0,9.0,6.0,...,0.973492,0.997368,-0.238860,1.460963,-1.426755,-0.110373,-0.231747,0.248542,-0.477033,-0.603430
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8.0,3-Nov-15,10.0,428.0,9.0,4.0,...,0.119211,0.846649,-0.623961,1.028874,-0.802665,-0.543430,-0.046856,0.952382,-1.363902,0.311719
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9.0,27-Nov-16,37.0,669.0,0.0,2.0,...,-0.699128,0.923917,0.049894,0.836199,1.279051,-0.948974,-1.558695,0.919782,-0.166663,1.465009
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2.0,28-Nov-15,43.0,373.0,0.0,5.0,...,0.316690,-0.134260,-0.751471,0.827784,0.137164,-0.610082,0.514635,0.618781,-0.698736,0.404212


In [9]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [55]:
len(embedded_df)

11316

In [56]:
#This Neural Net is just going to focus on classifying the Condition Cluster by the Review Embeddings
#This Neural Net is also going to use the dataframe that has not had dimensions reduced

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [57]:
# I am going to drop the columns that are not embeddings of the review and the condition_cluster

columns_to_drop = ['uniqueID', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'lengthReview', 'drugNameCluster_label']

embedded_df.drop(columns=columns_to_drop, inplace=True)

In [58]:
#check the dataframe

embedded_df.head()

,conditionCluster_label,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,2.0,6.256752,0.089550,-15.717361,7.261350,0.713106,-1.665947,-5.037074,-1.290510,0.278264,...,0.028273,0.953118,-0.814256,1.995593,0.972909,-0.869742,-0.927267,1.195638,-0.330614,0.845055
1,9.0,-3.616027,-1.046721,-3.250082,-6.424401,1.245648,0.238435,1.832553,-2.585644,5.422115,...,0.973492,0.997368,-0.238860,1.460963,-1.426755,-0.110373,-0.231747,0.248542,-0.477033,-0.603430
2,9.0,-5.944920,-0.564254,-3.821374,-2.283092,-7.821428,-1.080980,1.283531,-4.533960,-0.411771,...,0.119211,0.846649,-0.623961,1.028874,-0.802665,-0.543430,-0.046856,0.952382,-1.363902,0.311719
3,0.0,1.624358,0.118632,-2.822628,-9.034973,-0.346689,1.570179,0.553305,-3.568284,0.877865,...,-0.699128,0.923917,0.049894,0.836199,1.279051,-0.948974,-1.558695,0.919782,-0.166663,1.465009
4,0.0,-4.455269,-1.187546,3.133660,-4.596779,-2.269232,-5.121815,-2.388083,3.836105,5.048254,...,0.316690,-0.134260,-0.751471,0.827784,0.137164,-0.610082,0.514635,0.618781,-0.698736,0.404212


In [59]:
#checking on why the values of the condition cluster look like a float

condition_value_counts = embedded_df['conditionCluster_label'].value_counts()
print(condition_value_counts)

0.0    4074
8.0    2100
2.0     955
1.0     865
7.0     777
9.0     752
4.0     736
5.0     356
3.0     353
6.0     347
Name: conditionCluster_label, dtype: int64


In [60]:
# When I start training the model I am getting an error messsage that NAN values are present
embedded_df = embedded_df.dropna(subset=['conditionCluster_label'])

In [61]:
len(embedded_df)

11315

In [62]:
# Split our preprocessed data into our features and target arrays
# Choosing randon_state = 30 to be able to re-create attempts later
# There is a lot of data, so I can train on 70%, setting test_size at .3
# This step is necessary for training the neural network to classify 10 categories
# This code is altered by chatgpt after I could not fix it on my own

from keras.utils import to_categorical

# Separate the target variable
target = embedded_df['conditionCluster_label']

# Verify the unique values in the target variable
print(target.unique())

# Extract features (excluding the target variable)
features = embedded_df.drop(columns=['conditionCluster_label'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=30)

# Convert target labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Check the shapes of the one-hot encoded target labels
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

[2. 9. 0. 1. 3. 8. 5. 7. 4. 6.]
Shape of y_train_encoded: (7920, 10)
Shape of y_test_encoded: (3395, 10)


In [63]:
print(y_train)

9616     7.0
9338     2.0
1737     7.0
11308    5.0
8582     3.0
        ... 
4859     2.0
919      2.0
500      5.0
4517     0.0
5925     7.0
Name: conditionCluster_label, Length: 7920, dtype: float64


In [66]:
# The Y-Train shape was (None, 1) and not (None, 10), this was causing error messages late on, I think this undoes the one-hot encoding

#import numpy as np

#y_train = np.c_[y_train, np.zeros((y_train.shape[0], 9))]

#print(X_train)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [68]:
# Check the number of dimensions, make it a variable so it passes into

num_dimensions = embedded_df.shape[1]-1
print(num_dimensions)

150


In [69]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Most of this is going to be the same as the HW assignment, but there are 10 classifications to predict now
# So the output layer has been adjusted

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="sigmoid", input_dim=num_dimensions))

# Second hidden layer
nn.add(Dense(units=16, activation='sigmoid'))

# Third hidden layer (example of adding an additional hidden layer)
nn.add(Dense(units=8, activation='sigmoid'))

#Fourth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='ELU'))

#Fifth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='leaky_relu'))

#Sixth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='relu'))

#Seventh hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='ELU'))

#Eighth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='relu'))

# Output layer
nn.add(Dense(units=10, activation='softmax'))  # 10 units for 10 classes, softmax activation

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 32)                4832      
                                                                 
 dense_25 (Dense)            (None, 16)                528       
                                                                 
 dense_26 (Dense)            (None, 8)                 136       
                                                                 
 dense_27 (Dense)            (None, 4)                 36        
                                                                 
 dense_28 (Dense)            (None, 4)                 20        
                                                                 
 dense_29 (Dense)            (None, 4)                 20        
                                                                 
 dense_30 (Dense)            (None, 4)                

In [70]:
# Compile the model - the loss function is categorical and not for binary classification

nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [72]:
 # Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=10)

Epoch 1/10
248/248 [==============================] - 2s 3ms/step - loss: 2.2379 - accuracy: 0.3574
Epoch 2/10
248/248 [==============================] - 1s 4ms/step - loss: 2.1336 - accuracy: 0.3600
Epoch 3/10
248/248 [==============================] - 1s 4ms/step - loss: 2.0624 - accuracy: 0.3600
Epoch 4/10
248/248 [==============================] - 1s 4ms/step - loss: 2.0164 - accuracy: 0.3600
Epoch 5/10
248/248 [==============================] - 1s 3ms/step - loss: 1.9885 - accuracy: 0.3600
Epoch 6/10
248/248 [==============================] - 1s 3ms/step - loss: 1.9726 - accuracy: 0.3600
Epoch 7/10
248/248 [==============================] - 1s 3ms/step - loss: 1.9641 - accuracy: 0.3600
Epoch 8/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9597 - accuracy: 0.3600
Epoch 9/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9574 - accuracy: 0.3600
Epoch 10/10
248/248 [==============================] - 1s 3ms/step - loss: 1.9563 - accuracy: 0.3600

In [73]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 0s 2ms/step - loss: 1.9457 - accuracy: 0.3602
Test Loss: 1.9456828832626343
Test Accuracy: 0.36023563146591187


In [48]:
# The model immediately overfits and then actual accuracy is around 36% for the test data
# which honestly is better than I thought it would it do

In [49]:
# Proceed on to the techniques that ASIM talked about:
# #1. L2 Normalization
# #2. GridSearch
# #3 Cross validation

In [74]:
#Using L2 Normalization
from tensorflow.keras import regularizers

# Define the model - deep neural net with L2 regularization
nn = Sequential()

# First hidden layer with L2 regularization
nn.add(Dense(units=32, activation="sigmoid", kernel_regularizer=regularizers.l2(0.01), input_dim=150))

# Second hidden layer with L2 regularization
nn.add(Dense(units=16, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Output layer
nn.add(Dense(units=10, activation='softmax'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=20)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 32)                4832      
                                                                 
 dense_34 (Dense)            (None, 16)                528       
                                                                 
 dense_35 (Dense)            (None, 10)                170       
                                                                 
Total params: 5530 (21.60 KB)
Trainable params: 5530 (21.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
248/248 [==============================] - 1s 2ms/step - loss: 2.3183 - accuracy: 0.3225
Epoch 2/20
248/248 [==============================] - 0s 2ms/step - loss: 2.0044 - accuracy: 0.3600
Epoch 3/20
248/248 [==============================] - 1s 2ms/step - loss: 1.9702 - a

In [75]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 3ms/step - loss: 1.9446 - accuracy: 0.3602
Test Loss: 1.9445868730545044
Test Accuracy: 0.36023563146591187


In [ ]:
#Ok, not overfit. Not accurate, but the accuracy of the train and test data is basically the same after a few tests

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install tensorflow

In [ ]:
pip install --upgrade tensorflow

In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0.post1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [ ]:
!pip install scikeras

In [ ]:
from scikeras.wrappers import KerasClassifier

In [ ]:
# GridSearchCV = Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

In [ ]:
#Hypertuning with Gridsearch

# Define the neural network model
def create_model(optimizer='adam', activation='relu', units=32):
    model = Sequential()
    model.add(Dense(units=units, activation=activation, input_dim=768))
    model.add(Dense(units=16, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define hyperparameters grid
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['relu', 'sigmoid'],
    'units': [16, 32, 64]
}

# Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train_encoded)

# Print best parameters and best score
print("Best Parameters:", grid_result.best_params_)
print("Best Score:", grid_result.best_score_)

ValueError: Invalid parameter activation for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(activation=relu)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
#Skipping to cross-validation score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold


# Define the neural network model
def create_model():
    model = Sequential()
    model.add(Dense(units=32, activation='sigmoid', input_dim=768))
    model.add(Dense(units=16, activation='sigmoid'))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create KerasClassifier for cross-validation
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)

# Define K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform K-Fold Cross-Validation
results = cross_val_score(model, X_train_scaled, y_train_encoded, cv=kfold)

# Print average accuracy and standard deviation
print("Mean Accuracy:", results.mean())
print("Standard Deviation:", results.std())


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'